### Data Collection and Filtering

This file contians the code that gets the the data from the original excel docs, filters the data by removing unecessary data
such as irrelevant table columns and adding more financial data.
The added financial data includes: 
* SP500 weekly, monthly ,3 month percent change and daily SP500 gap percent change.
* Gap percent change of the given instrument.


In [95]:

import numpy as np
import pandas as pd
from financialData import financialData

import datetime

### Data Import:



In [96]:
dt1 = pd.read_excel(r'Live day trading 2 fixed.xlsx' , 'TABLE1')
dt2 = pd.read_excel(r'Live day trading 4.xlsx' , 'TABLE1')
dt3 = pd.read_excel(r'Live day trading 5.xlsx' , 'TABLE1')
dt4 = pd.read_excel(r'Live day trading 7.xlsx' , 'TABLE1')

### Data Wrangling:

1) Sign the tables, and remove their nulls under the "Instrument" column.

2) In dt3 and dt4 which are more recent data tables there has been a separation between 'Potential Price' and 'Potential Price in Trade', the latter is equal to 'Potential Price' in dt2 and d1 which determines the poteintial price a trade may reach without dropping back to SL. Same with 'RRR Potential in Trade' in dt3 and dt4 and 'RRR Potential' in dt2 and dt1.

3) Merge all 4 tables.


In [97]:
#1)

dt_all = [dt1,dt2,dt3,dt4]
table_num = 0

# sign the tables by enumeration
for table in dt_all:

    table["Table Number"] = table_num 
    table = table.dropna(subset=["Instrument"])
    table_num +=1

#2)
dt3 =  dt3.drop(['Potential Price', 'Trade Potential'], axis = 1) 
dt4 = dt4.drop(['Potential Price', 'Trade Potential'], axis = 1) 


dt3.rename(columns = {'In Trade Potential Price':'Potential Price'}, inplace = True) 
dt4.rename(columns = {'In Trade Potential Price':'Potential Price'}, inplace = True)

dt3.rename(columns = {'RRR Potential in Trade':'RRR Potential'}, inplace = True) 
dt4.rename(columns = {'RRR Potential in Trade':'RRR Potential'}, inplace = True)

#3)
dt = dt1.append(dt2,ignore_index=True)
dt = dt.append(dt3,ignore_index=True)
dt = dt.append(dt4,ignore_index=True)

dt.sample(5)



,Instrument,No,Entry Date,Entry Time,Exit Time,Comissions,Gain/Loss,Quantity,Setup,Buy/Sell,...,Exit 3,lvl 2,Performance Grade,Pivot Description,SandR,Dilution,Highest Price2,Shares,Position Size,Time2
65,OR,66,2019-02-21 00:00:00,09:56:00,10:06:00,2.04,-21.00,150.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,OTLK,54,2019-05-17 00:00:00,10:54:00,11:22:00,14.52,-196.93,1428.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,CRMD,30,2019-01-16 00:00:00,10:09:00,10:11:00,3.04,-20.59,300.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,TROV,93,2019-03-08 00:00:00,11:14:00,11:16:00,2.01,29.50,50.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,POAI,30,2020-02-05 00:00:00,10:00:00,10:04:00,NaN,NaN,NaN,BO,BUY,...,4.3,NaN,0.0,NaN,NaN,True,0.088889,66.0,313.5,00:04:00


### Filtering:



1) Choose only the given features to see if there is a relationship between them and the outcome of a trade.

2) Select only long pattern breakout setups.

3) Drop short trades, due to their irrelevance. This analysis only features long trades.





In [98]:
#1)
dt = dt[["Instrument", "Entry Date","Entry Time","Exit Time","Setup","Buy/Sell","Intended Entry","Entry Price"
,"SL Price", "Exit Price", "Highest Price", "Potential Price","Volume Exit","Wick Exit","Price Behaviour","Sector"
, "Catalyst", "Pattern", "Float","RRR in-trade","Negative RRR in-trade","Time","Outcome","Missed RRR on Entry"
,"Missed RRR","RRR Potential","Hard RRR Potential","RRR Difference","RRR Realized","RRR Volume Exit","RRR Wick Exit"
,"RRR Joint Wick and Volume Exit","Pause Num","VWAP Tag","VWAP","Table Number"]]

#2)
setups = ['BO','BOT','VBO','FPH','VF']

dt = dt[(dt['Buy/Sell'] == "BUY") & (dt['Setup'].isin(setups))].reset_index(drop=True)

#3)

# Buy/Sell column is now irrelevant because all positions in this dataset are Buy
# Setup column is also irrelevant because all setups are Break Outs or a subgroup of a Break Out trade.
dt =  dt.drop(['Buy/Sell','Setup'], axis =1)


### Extracting Additional Financial Data:

With the help of the fdata module which includes the financialData class, get the following from Yahoo Finance:

1) SP500 weekly, monthly ,3 month percent change and the daily gap.

2) The percent change gap of the given symbol at the date the trade took place.

In [99]:

# 1)

#create a new financialData object
SP500 = financialData('SPY')

week = 7
month = 30
three_months = 90


# apply the function from the fdata module and add the data to dt
dt['SPY Week Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), week) )
dt['SPY Month Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), month) )
dt['SPY 3 Month Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), three_months) )
dt['SPY Gap'] = dt['Entry Date'].apply(lambda x: SP500.getGap(x.date()) )



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [100]:
# 2)

# import the info for delisted stock from a manualy collected
#local excel file, because the delisted stocks are not presented in Yahoo Finance anymore.
delisted = pd.read_excel(r'Delisted.xlsx' )
delisted['Date']=delisted['Date'].apply(lambda x: x.date())

#extract the data from fdata module (Yahoo Finance) with the 'financialData' class, and from the 'delisted' table
def gap (instrument,entry_date):

    gap = 0
    if instrument not in delisted['Instrument'].values :
        print(instrument)
        data = financialData(instrument)
        gap= data.getGap(entry_date)
        print(gap)
    else:
        print("delisted: " + instrument)
        gap = delisted[(delisted['Date']==entry_date) & (delisted['Instrument']==instrument)]['Gap']
        gap = gap.values[0]*100

    return gap
  
dt['Gap'] = dt.apply(lambda x: gap(x['Instrument'], x['Entry Date'].date()), axis=1)



delisted: YECO
delisted: YECO
delisted: ALQA
INPX
[*********************100%***********************]  1 of 1 completed
38.79598347401169
delisted: DOVA
delisted: ADIL
delisted: CCCL
delisted: MRT
delisted: CCCL
delisted: FLKS
delisted: CCCL
RHE
[*********************100%***********************]  1 of 1 completed
0.7812492724042223
INPX
[*********************100%***********************]  1 of 1 completed
29.87804134221927
APHA
[*********************100%***********************]  1 of 1 completed
-2.279636734143715
UXIN
[*********************100%***********************]  1 of 1 completed
3.0303074089730497
INSG
[*********************100%***********************]  1 of 1 completed
0.8865282255054283
TYME
[*********************100%***********************]  1 of 1 completed
8.943090455232165
CENX
[*********************100%***********************]  1 of 1 completed
4.049079008000802
CRMD
[*********************100%***********************]  1 of 1 completed
5.29100539779674
delisted: BSTI
delist

In [101]:
#get the stock's daily open of a given trade
def open_price (instrument,entry_date):

    open_price = 0
    if instrument not in delisted['Instrument'].values :  
        print(instrument)
        data = financialData(instrument)
        open_price = data.getOpen(entry_date)
        print(open_price)
    else:
        print("delisted: " + instrument )
        open_price = delisted[(delisted['Date']==entry_date) & (delisted['Instrument']==instrument)]['Open']
        open_price = open_price.values[0]

    return open_price
  
dt['Daily Open'] = dt.apply(lambda x: open_price(x['Instrument'], x['Entry Date'].date()), axis=1)

delisted: YECO
delisted: YECO
delisted: ALQA
INPX
[*********************100%***********************]  1 of 1 completed
INPX
4.15
delisted: DOVA
delisted: ADIL
delisted: CCCL
delisted: MRT
delisted: CCCL
delisted: FLKS
delisted: CCCL
RHE
[*********************100%***********************]  1 of 1 completed
RHE
1.2899999618530273
INPX
[*********************100%***********************]  1 of 1 completed
INPX
4.25999993218316
APHA
[*********************100%***********************]  1 of 1 completed
APHA
6.429999828338623
UXIN
[*********************100%***********************]  1 of 1 completed
UXIN
3.4000000953674316
INSG
[*********************100%***********************]  1 of 1 completed
INSG
5.690000057220459
TYME
[*********************100%***********************]  1 of 1 completed
TYME
2.680000066757202
CENX
[*********************100%***********************]  1 of 1 completed
CENX
8.479999542236328
CRMD
[*********************100%***********************]  1 of 1 completed
CRMD
1.98999996

In [102]:
dt[['Intended Entry','Daily Open','VWAP']]

,Intended Entry,Daily Open,VWAP
0,5.30,4.90,5.10
1,6.12,4.90,5.80
2,3.26,2.88,3.06
3,4.92,4.15,4.77
4,6.75,6.18,6.47
...,...,...,...
145,1.62,1.40,1.55
146,1.15,0.90,1.15
147,0.53,0.37,0.50
148,4.75,2.90,4.50


### Round Floats:


Round all float values to maximum 2 numbers after the decimal point. 


In [104]:
# list of columns with numeric values
num_colums = ["Intended Entry","Entry Price","SL Price", "Exit Price", "Highest Price", "Potential Price","Volume Exit"
,"Wick Exit","Price Behaviour", "Float","RRR in-trade","Negative RRR in-trade","Time","Missed RRR on Entry","Missed RRR"
,"RRR Potential","Hard RRR Potential","RRR Difference","RRR Realized","RRR Volume Exit","RRR Wick Exit"
,"RRR Joint Wick and Volume Exit","VWAP Location R","Table Number","Gap","Daily Open","SPY Week Change","SPY Month Change","SPY 3 Month Change"
, "VWAP"]


# special roud up function that also cuts unnecessary zeros
def roundUp (x):
    if isinstance(x, float):
        x = np.around(x,2)
        s = str(x)
        s.rstrip('0').rstrip('.') if '.' in s else s
        x = float(s)
    return x

# round up
for col in num_colums:
    dt[col] = dt[col].apply(lambda x: roundUp(x))

### Export to xlsx:



In [106]:
dt.to_excel("data.xlsx")